In [ ]:
from sklearn.neighbors import NearestNeighbors, kneighbors_graph
from sklearn.manifold import trustworthiness
from scipy.sparse.csgraph import dijkstra
from scipy.spatial import distance_matrix
from sklearn.manifold import MDS
import matplotlib.pyplot as plt
import multiprocessing as mp
from tqdm.auto import tqdm
import tensorflow as tf
import numpy as np
import random
import queue
import json


# Antenna definitions
ASSIGNMENTS = [
	[0, 13, 31, 29, 3, 7, 1, 12 ],
	[30, 26, 21, 25, 24, 8, 22, 15],
	[28, 5, 10, 14, 6, 2, 16, 18],
	[19, 4, 23, 17, 20, 11, 9, 27]
]

ANTENNACOUNT = np.sum([len(antennaArray) for antennaArray in ASSIGNMENTS])

# Optimized version: Calibration constants precomputed once
def load_calibrate_timedomain(path, offset_path):
    with open(offset_path, "r") as offsetfile:
        offsets = json.load(offsetfile)

    # Precompute the calibration tensors
    fft_len = 1024
    freq_range = 2 * np.pi * np.arange(fft_len, dtype=np.float32) / fft_len
    sto_offset = tf.constant(offsets["sto"], dtype=tf.float32)[:, None] * freq_range
    cpo_offset = tf.constant(offsets["cpo"], dtype=tf.float32)[:, None]
    calib_multiplier = tf.exp(tf.complex(0.0, sto_offset + cpo_offset))

    def record_parse_function(proto):
        record = tf.io.parse_single_example(proto, {
            "csi": tf.io.FixedLenFeature([], tf.string),
            "pos-tachy": tf.io.FixedLenFeature([], tf.string),
            "time": tf.io.FixedLenFeature([], tf.float32),
        })

        csi = tf.io.parse_tensor(record["csi"], out_type=tf.float32)
        csi = tf.complex(csi[:, :, 0], csi[:, :, 1])
        csi = tf.signal.fftshift(csi, axes=1)

        position = tf.io.parse_tensor(record["pos-tachy"], out_type=tf.float64)
        time = record["time"]
        return csi, position[:2], time

    def apply_calibration(csi, pos, time):
        csi = tf.multiply(csi, calib_multiplier)
        return csi, pos, time

    def csi_time_domain(csi, pos, time):
        csi = tf.signal.fftshift(tf.signal.ifft(tf.signal.fftshift(csi, axes=1)), axes=1)
        return csi, pos, time

    def cut_out_taps(tap_start, tap_stop):
        def func(csi, pos, time):
            return csi[:, tap_start:tap_stop], pos, time
        return func

    def order_by_antenna(csi, pos, time):
        csi = tf.stack([tf.gather(csi, indices) for indices in ASSIGNMENTS])
        return csi, pos, time

    dataset = tf.data.TFRecordDataset(path)
    dataset = dataset.map(record_parse_function, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.map(apply_calibration, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.map(csi_time_domain, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.map(cut_out_taps(507, 520), num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.map(order_by_antenna, num_parallel_calls=tf.data.AUTOTUNE)

    return dataset

inputpaths = [
	{
		"tfrecords" : "dichasus/dichasus-cf02.tfrecords",
		"offsets" : "dichasus/reftx-offsets-dichasus-cf02.json"
	},
	{
		"tfrecords" : "dichasus/dichasus-cf03.tfrecords",
		"offsets" : "dichasus/reftx-offsets-dichasus-cf03.json"
	},
	{
		"tfrecords" : "dichasus/dichasus-cf04.tfrecords",
		"offsets" : "dichasus/reftx-offsets-dichasus-cf04.json"
	}
]

full_dataset = load_calibrate_timedomain(inputpaths[0]["tfrecords"], inputpaths[0]["offsets"])

for path in inputpaths[1:]:
	full_dataset = full_dataset.concatenate(load_calibrate_timedomain(path["tfrecords"], path["offsets"]))


In [ ]:
groundtruth_positions = []
csi_time_domain = []
timestamps = []
for csi, pos, time in full_dataset.batch(1000):
	csi_time_domain.append(csi.numpy())
	groundtruth_positions.append(pos.numpy())
	timestamps.append(time.numpy())

csi_time_domain = np.concatenate(csi_time_domain)
groundtruth_positions = np.concatenate(groundtruth_positions)
timestamps = np.concatenate(timestamps)

In [ ]:
csi_time_domain.shape

In [ ]:
def feature_extraction_F1(csi_matrix):
    csi_ifft = np.fft.ifft(csi_matrix, axis=1)
    R = csi_ifft * np.conj(csi_ifft)
    r = R.flatten()
    r_real = np.real(r)
    r_imag = np.imag(r)
    r_R = np.concatenate((r_real, r_imag))
    feature_vector = r_R / np.linalg.norm(r_R)
    return feature_vector
m=csi_time_domain.shape[0]
reshaped_csi = csi_time_domain.reshape(m, 32, 13)
features_F1 = np.array([feature_extraction_F1(csi) for csi in reshaped_csi])
print("Shape of Features (F1):", features_F1.shape)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, x1, y_train, y1 = train_test_split(features_F1, groundtruth_positions, test_size=0.4, random_state=42)
X_test,X_pred, y_test, y_pred = train_test_split(x1, y1, test_size=0.5, random_state=42)
print("Training features shape:", X_train.shape)
print("Testing features shape:", X_test.shape)
print("Training positions shape:", y_train.shape)
print("Testing positions shape:", y_test.shape)

In [ ]:
pos_x_train= y_train[:, 0].reshape(-1,1)
pos_y_train= y_train[:, 1].reshape(-1,1)
pos_x_test= y_test[:, 0].reshape(-1,1)
pos_y_test= y_test[:, 1].reshape(-1,1)
pos_x_pred= y_pred[:, 0].reshape(-1,1)
pos_y_pred= y_pred[:, 1].reshape(-1,1)

In [ ]:
import numpy as np
from scipy.special import softmax

def build_grid(grid_size=30, space_bounds=None):
    (xmin, xmax), (ymin, ymax) = space_bounds
    x_lin = np.linspace(xmin, xmax, grid_size)
    y_lin = np.linspace(ymin, ymax, grid_size)
        
    xv, yv = np.meshgrid(x_lin, y_lin)
    grid_points = np.stack([xv.ravel(), yv.ravel()], axis=1)
    return grid_points

def compute_soft_probability_maps(positions, grid_points, temperature=1.0):
    N = positions.shape[0]
    K = grid_points.shape[0]
    dists = np.linalg.norm(positions[:, np.newaxis, :] - grid_points[np.newaxis, :, :], axis=2)
    sim_scores = -dists / temperature
    prob_maps = softmax(sim_scores, axis=1)
    return prob_maps

def estimate_positions_from_maps(prob_maps, grid_points):
    return np.matmul(prob_maps, grid_points)

def generate_all_probability_maps(positions, grid_size=30, temperature=1.0):
    x_min, x_max = positions[:, 0].min(), positions[:, 0].max()
    y_min, y_max = positions[:, 1].min(), positions[:, 1].max()

    grid_points = build_grid(grid_size, space_bounds=((x_min, x_max), (y_min, y_max)))
    print("Grid points shape:", grid_points.shape)
    prob_maps = compute_soft_probability_maps(positions, grid_points, temperature=temperature)

    est_locations = estimate_positions_from_maps(prob_maps, grid_points)

    return prob_maps, est_locations, grid_points

prob_maps, est_positions, grid_points = generate_all_probability_maps(y_train)

print(prob_maps.shape)
print(est_positions.shape)


In [ ]:
(y_train-est_positions).std(axis=0)

In [ ]:
from keras import layers

class NeuralNetwork(tf.keras.Model):
    def __init__(self, input_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = layers.Dense(2048, activation='relu', input_dim=input_size)
        self.bn1 = layers.BatchNormalization()
        self.fc2 = layers.Dense(2048, activation='relu')
        self.bn2 = layers.BatchNormalization()
        self.fc3 = layers.Dense(2048, activation='relu')
        self.fc4 = layers.Dense(2048, activation='relu')
        self.fc5 = layers.Dense(2048, activation='relu')
        # Output layer: output_size = number of grid points, softmax for probability
        self.fc6 = layers.Dense(output_size, activation='softmax')

    def call(self, inputs):
        x = self.fc1(inputs)
        x = self.bn1(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        x = self.fc5(x)
        x = self.fc6(x)
        return x

# Set output_size to the number of grid points (e.g., 900 for 30x30 grid)
output_size = grid_points.shape[0]  # grid_points from your earlier code
model = NeuralNetwork(input_size=832, output_size=output_size)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',  # or 'kullback_leibler_divergence'
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, prob_maps, epochs=30, batch_size=32, validation_split=0.2)

In [ ]:
predicted_maps = model.predict(X_test)
predicted_positions = np.matmul(predicted_maps, grid_points)
mae = np.mean(np.abs(y_test - predicted_positions))
print("Mean Absolute Error (Position Predictions):", mae)

In [ ]:
import tensorflow as tf
from keras.layers import Input, Dense, BatchNormalization, Dropout
from keras.models import Model

def build_denoising_autoencoder(input_dim):
    """
    Builds a denoising autoencoder model.

    Args:
        input_dim: Dimension of the input features (e.g., 832 for features_F1).

    Returns:
        autoencoder: The denoising autoencoder model.
    """
    # Input layer
    input_data = Input(shape=(input_dim,), name="input")

    # Encoder
    x = Dense(512, activation="relu", name="encoder_dense_1")(input_data)
    x = BatchNormalization(name="encoder_bn_1")(x)
    x = Dropout(0.2, name="encoder_dropout_1")(x)

    x = Dense(256, activation="relu", name="encoder_dense_2")(x)
    x = BatchNormalization(name="encoder_bn_2")(x)
    x = Dropout(0.2, name="encoder_dropout_2")(x)

    x = Dense(128, activation="relu", name="encoder_dense_3")(x)
    x = BatchNormalization(name="encoder_bn_3")(x)
    x = Dropout(0.2, name="encoder_dropout_3")(x)

    x = Dense(64, activation="relu", name="encoder_dense_4")(x)
    x = BatchNormalization(name="encoder_bn_4")(x)
    x = Dropout(0.2, name="encoder_dropout_4")(x)

    encoded = Dense(32, activation="relu", name="encoder_latent")(x)  # Latent space

    # Decoder
    x = Dense(64, activation="relu", name="decoder_dense_1")(encoded)
    x = BatchNormalization(name="decoder_bn_1")(x)
    x = Dropout(0.2, name="decoder_dropout_1")(x)

    x = Dense(128, activation="relu", name="decoder_dense_2")(x)
    x = BatchNormalization(name="decoder_bn_2")(x)
    x = Dropout(0.2, name="decoder_dropout_2")(x)

    x = Dense(256, activation="relu", name="decoder_dense_3")(x)
    x = BatchNormalization(name="decoder_bn_3")(x)
    x = Dropout(0.2, name="decoder_dropout_3")(x)

    x = Dense(512, activation="relu", name="decoder_dense_4")(x)
    x = BatchNormalization(name="decoder_bn_4")(x)
    x = Dropout(0.2, name="decoder_dropout_4")(x)

    decoded = Dense(input_dim, activation="linear", name="decoder_output")(x)  # Reconstruct clean features

    # Autoencoder model
    autoencoder = Model(input_data, decoded, name="DenoisingAutoencoder")

    return autoencoder

# Build the denoising autoencoder
input_dim = 832  # Dimension of the input features
autoencoder = build_denoising_autoencoder(input_dim)

# Compile the autoencoder
autoencoder.compile(optimizer="adam", loss="mse")

# Print the autoencoder summary
autoencoder.summary()

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# from scipy.signal import convolve

# mae_noise_list = []
# mae_denoise_list = []
# lp_values = []

# groundtruth_maps, est_positions, grid_points = generate_all_probability_maps(groundtruth_positions)
# for x in range(1, 10):
#     lp_values.append(x)

#     # Parameters
#     W = 1024
#     modulation_order = 16
#     C = W // 8
#     L = 13
#     Lp = x
#     noise_power = 0.1
#     B = 32
#     # Generate frequency-domain symbols
#     data = np.random.randint(0, modulation_order, W)
#     qam_symbols = (2 * (data % 4) - 3) + 1j * (2 * (data // 4) - 3)
#     qam_symbols /= np.sqrt(10)

#     time_domain_signal = np.fft.ifft(qam_symbols)

#     # Zero-mean Gaussian for amplitude
#     A_k = np.random.normal(loc=0.0, scale=1.0, size=Lp)
#     # Zero-mean Gaussian for phase
#     phi_k = np.random.normal(loc=0.0, scale=1.0, size=Lp)

#     p_bar = A_k * np.exp(1j * phi_k)
#     p_bar /= np.linalg.norm(p_bar)

#     perturbed_signal = np.convolve(p_bar, time_domain_signal, mode="full")[:W]
#     cyclic_prefix = perturbed_signal[-C:]
#     transmit_signal = np.concatenate([cyclic_prefix, perturbed_signal])

#     csi_time_domain = csi_time_domain.reshape(83403, 32, 13)
#     num_csi_instances = csi_time_domain.shape[0]

#     y_real_all = np.zeros((num_csi_instances, B, W + C), dtype=complex)
#     H_pert_all = np.zeros((num_csi_instances, B, W), dtype=complex)

#     for i in range(num_csi_instances):
#         H_real = csi_time_domain[i]
#         for b in range(B):
#             y_real = convolve(H_real[b], transmit_signal, mode='full')[:W + C]
#             noise = np.sqrt(noise_power / 2) * (np.random.randn(W + C) + 1j * np.random.randn(W + C))
#             y_real_noisy = y_real + noise
#             y_real_no_cp = y_real_noisy[C:]

#             y_freq = np.fft.fft(y_real_no_cp)
#             s_freq = np.fft.fft(time_domain_signal)

#             H_pert_all[i, b, :] = y_freq / s_freq
#             H_pert_time = np.fft.ifft(H_pert_all[i, b, :])[:L]
#             csi_time_domain[i, b, :] = H_pert_time

#     csi_time_domain = csi_time_domain.reshape(83403, 4, 8, 13)
#     reshaped_csi = csi_time_domain.reshape(83403, 32, 13)
#     features_F1 = np.array([feature_extraction_F1(csi) for csi in reshaped_csi])

#     x_noise = model.predict(features_F1)

#     mae_noise = np.mean(np.abs(x_noise - groundtruth_maps))

#     mae_noise_list.append(mae_noise)

# # Plot MAE vs Perturbation Length
# plt.plot(lp_values, mae_noise_list, label='MAE (Noisy Features)', marker='o')
# plt.xlabel('Perturbation Length (Lp)')
# plt.ylabel('Mean Absolute Error (MAE)')
# plt.title('MAE vs Perturbation Length')
# plt.legend()
# plt.grid(True)
# plt.show()

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import convolve

mae_noise_list = []
lp_values = []

# Constants
MAX_SAMPLES = 10000
BATCH_SIZE = 1024
AUTOTUNE = tf.data.AUTOTUNE

# Preprocess data
groundtruth_maps, est_positions, grid_points = generate_all_probability_maps(groundtruth_positions)

# Reduce data size
csi_time_domain = csi_time_domain[:MAX_SAMPLES]
groundtruth_maps = groundtruth_maps[:MAX_SAMPLES]

for x in range(1, 200):
    lp_values.append(x)

    # Parameters
    W = 1024
    modulation_order = 16
    C = W // 8
    L = 13
    Lp = x
    noise_power = 0.1
    B = 32

    # QAM generation
    data = np.random.randint(0, modulation_order, W)
    qam_symbols = (2 * (data % 4) - 3) + 1j * (2 * (data // 4) - 3)
    qam_symbols /= np.sqrt(10)
    time_domain_signal = np.fft.ifft(qam_symbols)

    # Perturbation generation
    A_k = np.random.normal(0.0, 1.0, size=Lp)
    phi_k = np.random.normal(0.0, 1.0, size=Lp)
    p_bar = A_k * np.exp(1j * phi_k)
    p_bar /= np.linalg.norm(p_bar)

    perturbed_signal = np.convolve(p_bar, time_domain_signal, mode="full")[:W]
    cyclic_prefix = perturbed_signal[-C:]
    transmit_signal = np.concatenate([cyclic_prefix, perturbed_signal])

    # Reshape CSI
    csi_time_domain = csi_time_domain.reshape(MAX_SAMPLES, 32, 13)

    # Perturb CSI
    for i in range(MAX_SAMPLES):
        H_real = csi_time_domain[i]
        for b in range(B):
            y_real = convolve(H_real[b], transmit_signal, mode='full')[:W + C]
            noise = np.sqrt(noise_power / 2) * (np.random.randn(W + C) + 1j * np.random.randn(W + C))
            y_real_noisy = y_real + noise
            y_real_no_cp = y_real_noisy[C:]

            y_freq = np.fft.fft(y_real_no_cp)
            s_freq = np.fft.fft(time_domain_signal)

            H_freq = y_freq / s_freq
            H_time = np.fft.ifft(H_freq)[:L]
            csi_time_domain[i, b, :] = H_time

    # Reshape for feature extraction
    csi_time_domain = csi_time_domain.reshape(MAX_SAMPLES, 4, 8, 13)
    reshaped_csi = csi_time_domain.reshape(MAX_SAMPLES, 32, 13)

    # Feature extraction
    features_F1 = np.array([feature_extraction_F1(csi) for csi in reshaped_csi])

    # Convert to TensorFlow Dataset
    dataset = tf.data.Dataset.from_tensor_slices(features_F1)
    dataset = dataset.batch(BATCH_SIZE).prefetch(AUTOTUNE)

    # Predict in batches using GPU
    x_noise = model.predict(dataset, verbose=0)

    # Compute MAE
    mae_noise = np.mean(np.abs(x_noise - groundtruth_maps))
    mae_noise_list.append(mae_noise)

# Plotting
plt.plot(lp_values, mae_noise_list, label='MAE (Noisy Features)', marker='o')
plt.xlabel('Perturbation Length (Lp)')
plt.ylabel('Mean Absolute Error (MAE)')
plt.title('MAE vs Perturbation Length')
plt.legend()
plt.grid(True)
plt.show()
